# Imports

In [4]:
import dijkstra
import astar
import pandas
import data_loader
import timer
import cli_utils

# Loading data

In [5]:
graph= data_loader.load_data_to_graph(normalize_coordinates=True)
graph_dict = graph.graph_dict

# Test data

In [6]:
bus_stops = [
    ('wrocławski park przemysłowy','pl. grunwaldzki'),
    ('grabiszyńska','pl. daniłowskiego'),
    ('kwiska','pl. grunwaldzki'),
    ('krzyki','leśnica'),
    ('leśnica','biskupin'),
    ('wrocławski park przemysłowy','dworzec autobusowy'),
    ('fat','osiedle sobieskiego')
]
bus_stops_test_data = [(graph_dict[start_stop], graph_dict[end_stop]) for start_stop, end_stop in bus_stops]

start_times = ['9:00','9:15','9:30', '2:00', '2:15', '2:30']
start_times_test_data = [cli_utils.normalize_input_time(time) for time in start_times]

# Data analysis

In [8]:
data = []
configs = {
    'Dijkstra for time' : lambda start_stop, end_stop, start_time: dijkstra.shortest_path(graph_dict=graph_dict, start_stop=start_stop, goal_stop=end_stop, start_time=start_time, cost_fn=dijkstra.DEFAULT_COST_FUNCTION),
    'A* for time' : lambda start_stop, end_stop, start_time: astar.astar(start=start_stop, goal=end_stop, start_time=start_time, cost_fn=astar.TIME_COST_FUNCTION, heuristic_fn=astar.TIME_HEURISTIC),
    'A* for line changes' : lambda start_stop, end_stop, start_time: astar.astar(start=start_stop, goal=end_stop, start_time=start_time, cost_fn=astar.LINE_CHANGE_COST_FUNCTION, heuristic_fn=astar.LINE_CHANGE_HEURISTIC)
}

timer = timer.Timer()

for start_time in start_times_test_data:
    for start_stop, end_stop in bus_stops_test_data:
        for config_name, function in configs.items():
            
            cost, _ = timer.run(lambda : function(start_stop, end_stop, start_time))
            computation_time = timer.elapsed_time
            
            record = config_name, start_stop, end_stop, start_time, cost, computation_time
            data.append(record)